## 基金代码名称对照表

Alex

Created on: 08/07/2021

In [1]:
import json
import time
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import os 
import warnings

warnings.filterwarnings('ignore')


# 首先需要更改 Chrome_Driver_Path 为你的本地路径
# 如果之前没有接触过Selenium可以参考下面这个链接
# https://github.com/RuichongWang/Web-Scraping/blob/main/Tutorials/Selenium%20Tutorial%20-%20Auto%20Searching.md
Chrome_Driver_Path='/Users/user/Downloads/网页下载/chromedriver' 

if not os.path.exists('Data'):
    os.mkdir('Data')

In [2]:
# 首先，去天天基金网下载全部公募基金主题代码对照表
# 用于之后根据基金行业筛选
url='http://fund.eastmoney.com/daogou/'
driver = webdriver.Chrome(Chrome_Driver_Path)
driver.get(url)
driver.find_element_by_xpath('//*[@id="jjztfoldType"]').click()
res=BeautifulSoup(driver.page_source)
driver.quit()

cat_all=res.find_all('dd',attrs={"id":"dd_tp"})+res.find_all('dd',attrs={"id":"zzjjafloat"})
cat_all=[x.find_all('a') for x in cat_all]
cat_all=sum(cat_all,[])
cat_all=[[x.get_text(),x.get('id')] for x in cat_all]
cat_df=pd.DataFrame(cat_all,columns=['板块','后缀'])

display(cat_df)
cat_df.to_csv('Data/主题代码对照表.csv',index=False)

,板块,后缀
0,工程建设,tp_BK0425
1,交运设备,tp_BK0429
2,农牧饲渔,tp_BK0433
3,煤炭采选,tp_BK0437
4,食品饮料,tp_BK0438
...,...,...
105,网红直播,tp_BK0940
106,无线充电,tp_BK0960
107,航天概念,tp_BK0963
108,生物识别,tp_BK0970


In [3]:
%%time
# 然后去网易财经爬取全部指数型基金的名称和代码对照表
result=[]
for i in range(10):
    url='http://quotes.money.163.com/fn/service/netvalue.php?host=/fn/service/netvalue.php&page=%s&query=STYPE:FDO;TYPE3:ZSX&fields=no,PUBLISHDATE,SYMBOL,SNAME,NAV,PCHG,M12RETRUN,SLNAVG,ZJZC&sort=PCHG&order=desc&count=500'%i
    res=requests.get(url)
    res=res.text
    res=json.loads(res)['list']
    res=[[x['SNAME'],x['SYMBOL']] for x in res]
    result+=res
    time.sleep(1)

 
result=pd.DataFrame(result,columns=['基金名称','基金代码'])
display(result)
result.to_csv('Data/指数型基金名称代码对照表.csv',index=False,encoding='utf-8 sig')


In [ ]:
# 去天天基金网爬取混合型基金名称
driver = webdriver.Chrome(Chrome_Driver_Path)
driver.get('http://fund.eastmoney.com/allfund.html')
text=driver.page_source
bs=BeautifulSoup(text)
chunks=bs.find_all('div',attrs={'class':'num_box'})
driver.quit()

res=[]
for chunk in chunks:
    res.extend(chunk.text.replace( ' | 基金吧 | 档案','').split('（'))

res=list(map(lambda x:x.split('）'),res))
res=pd.DataFrame(res,columns=['基金代码','基金名称']).dropna().reset_index(drop=True)
display(res)
res.to_csv('Data/基金代码名称对照表.csv',index=False)

,基金代码,基金名称
0,000001,华夏成长混合
1,000003,中海可转债债券A
2,000004,中海可转债债券C
3,000005,嘉实增强信用定期债券
4,000006,西部利得量化成长混合A
...,...,...
13387,970062,华鑫乐享周周购三个月持有C
13388,970067,兴证资管金麒麟消费升级混合
13389,970068,兴证资管金麒麟消费升级混合
13390,970069,兴证资管金麒麟消费升级混合C
